In [6]:
import os
from agno.agent import Agent
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.vectordb.qdrant import Qdrant
from agno.models.openai import OpenAIChat
from agno.document.chunking.recursive import RecursiveChunking
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
# Shared Qdrant collection (all chunks go here)
COLLECTION_NAME = "shared_user_docs"
QDRANT_URL = "http://localhost:6333"

def create_user_agent(user_id: str):
    user_pdf_path = f"data/{user_id}"  # Folder with this user's PDFs

    if not os.path.exists(user_pdf_path):
        raise FileNotFoundError(f"No document folder found for user_id: {user_id}")

    vector_db = Qdrant(
        collection=COLLECTION_NAME,
        url=QDRANT_URL
    )

    knowledge_base = PDFKnowledgeBase(
        path=user_pdf_path,
        vector_db=vector_db,
        reader=PDFReader(chunk=True),
        # chunking_strategy=RecursiveChunking(chunk_size=800, overlap=100),
        metadata={"user_id": user_id}  # Tag all chunks on ingestion
    )

    agent = Agent(
        model=OpenAIChat(id="gpt-3.5-turbo-0125"),
        knowledge=knowledge_base,
        user_id=user_id,  # Used internally by Agno to scope memory, RAG, history
        search_knowledge=True,
        instructions=[
            "You are a helpful assistant that answers questions based only on the user's uploaded PDF documents.",
            "Cite the document name if possible.",
            "If the answer isn't in the documents, respond clearly with 'I couldn't find that in your files.'"
        ]
    )

    return agent, knowledge_base


In [9]:
if __name__ == "__main__":
    user_id = input("Enter your user ID: ").strip()

    try:
        agent, kb = create_user_agent(user_id)

        # First-time document loading (or after new uploads)
        kb.load(recreate=False)

        print(f"🤖 Agent ready for user `{user_id}`. Ask your questions.")
        while True:
            query = input("\nYour question (or 'exit'): ")
            if query.lower() in {"exit", "quit"}:
                break
            response = agent.run(query)
            print("🧠", response.content)

    except FileNotFoundError as e:
        print("❌", e)


INFO Embedder not provided, using OpenAIEmbedder as default.

TypeError: cannot unpack non-iterable Agent object

In [4]:
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.document.chunking.recursive import RecursiveChunking
from agno.vectordb.qdrant import Qdrant

def ingest_user_documents(user_id: str):
    knowledge_base = PDFKnowledgeBase(
        path=f"data/{user_id}",                         # ✅ folder where PDFs for that user are stored
        vector_db=Qdrant(
            collection="multi_user_docs",
            url="http://localhost:6333"
        ),
        reader=PDFReader(chunk=True),
        # chunking_strategy=RecursiveChunking(chunk_size=800, overlap=100),
        metadata={"user_id": user_id}                         # ✅ CRUCIAL: this tags all chunks with user_id
    )
    
    # Load documents only if new or not already indexed
    knowledge_base.load(recreate=False)


In [5]:
from agno.agent import Agent
from agno.knowledge import AgentKnowledge
from agno.embedder.openai import OpenAIEmbedder
from agno.models.openai import OpenAIChat
from agno.vectordb.qdrant import Qdrant as AgnoQdrant

def create_user_agent(user_id: str):
    vector_db = AgnoQdrant(
        collection="multi_user_docs",
        url="http://localhost:6333"
    )

    knowledge = AgentKnowledge(
        vector_db=vector_db,
        embedder=OpenAIEmbedder(),
        user_id=user_id  # ✅ Agno will now only retrieve chunks for this user
    )

    agent = Agent(
        model=OpenAIChat(id="gpt-4.1-nano"),
        knowledge=knowledge,
        user_id=user_id,
        search_knowledge=True,
        instructions=[
            "You are a helpful assistant that only uses the user's uploaded documents.",
            "Cite the file when relevant, and be honest when the answer isn't found."
        ]
    )
    return agent


In [8]:
if __name__ == "__main__":
    user_id = input("Enter your user ID: ").strip()

    # Step 1: Ingest and tag user documents (run only once or when PDFs change)
    ingest_user_documents(user_id)

    # Step 2: Create isolated agent
    agent = create_user_agent(user_id)

    # Step 3: Chat loop
    while True:
        query = input(f"[{user_id}] Ask your agent: ")
        if query.lower() in {"exit", "quit"}:
            break
        response = agent.run(query)
        print("🤖", response.content)


INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Loading knowledge base

INFO Reading: Introduction

INFO Added 0 documents to knowledge base

INFO Embedder not provided, using OpenAIEmbedder as default.

🤖 Akreage is a comprehensive blockchain-powered ecosystem focused on real estate crowdfunding and investment. It utilizes its governance token called Akres, which enables holders to participate in project approvals, oversee project progress, and purchase Council NFTs that grant authority in the vetting process. The platform provides a transparent and regulated funding mechanism through its Akreage Launchpad, where developers raise funds in AUSD (Akreage Dollars) and projects establish investment funds in The Bahamas for legal and financial oversight. Additionally, Akreage offers a Payment System that facilitates seamless digital transactions within real estate developments, enhancing utility and efficiency for developers, investors, and other stakeholders. The ecosystem aims to democratize access to global real estate investments, combining blockchain technology with traditional funding structures.
